In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

# Import data

In [ ]:
g_data = pd.read_csv('poverty.txt', delim_whitespace=True, \
        index_col=[0])
g_data.head()

# Set up tensorflow graph

In [ ]:
class TF_regression:
    def __init__(self):
        self.sess = tf.Session()
        self._build_graph()
    
    
    def _build_graph(self):
        self.A = tf.placeholder(dtype=tf.float32)
        self.b = tf.placeholder(dtype=tf.float32)
        
        At = tf.transpose(self.A)
        AtA = tf.matmul(At, self.A)
        AtA_inv = tf.matrix_inverse(AtA)
        AtA_invAt = tf.matmul(AtA_inv, At)
        self.x = tf.matmul(AtA_invAt, self.b)
        
    
    def onedim_regression(self, data):
        self.sess.run(tf.global_variables_initializer())
        
        # Create feed dictionary. Careful with row and column vectors
        data_A = np.array(data.loc[:, 'PovPct'], ndmin=2).T
        ones_A = np.array(np.ones(data.shape[0]), ndmin=2).T
        input_A = np.hstack((data_A, ones_A))
        input_b = np.array(data.loc[:, 'Brth15to17'], ndmin=2).T
        feed_dict = {self.A: input_A, self.b: input_b}
        
        output = self.sess.run(self.x, feed_dict=feed_dict)
        
        print('Regression equation:')
        print('\'Brth15to17\' = %.2f + %.2f\'PovPct\'' % (output[0], output[1]))
    
        # Graph of regression
        X = np.arange(0, 31)
        Y = output[0]*X + output[1]
        fig, ax = plt.subplots()
        ax.scatter(g_data['PovPct'], g_data['Brth15to17'])    # Scatterplot of data
        ax.plot(X, Y, color='k')    # Plot of regression
        ax.set_xlabel('Poverty (%)')
        ax.set_ylabel('Births 15-17')
        ax.set_title('Births 15-17 vs. Poverty %')
        plt.savefig('regression_Brth15to17_PovPct.pdf')
        
        
    def twodim_regression(self, data):
        self.sess.run(tf.global_variables_initializer())
        
        # Create feed dictionary. Careful with row and column vectors
        data_A = np.array(data.loc[:, ['PovPct', 'ViolCrime']])
        ones_A = np.array(np.ones(data.shape[0]), ndmin=2).T
        input_A = np.hstack((data_A, ones_A))
        input_b = np.array(data.loc[:, 'Brth15to17'], ndmin=2).T
        feed_dict = {self.A: input_A, self.b: input_b}
    
        output = self.sess.run(self.x, feed_dict=feed_dict)
    
        # Print the regression equation
        print('Regression equation:')
        print('\'Brth15to17\' = %.2f + %.2f\'PovPct\' + %.2f\'ViolCrime\'' \
            % (output[0], output[1], output[2]))

# Run regressions

In [ ]:
model = TF_regression()
model.onedim_regression(g_data)
model.twodim_regression(g_data)